In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kamrannaseer","key":"86bfee909777b6a9a32c48d76ca589d5"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:19<00:00, 46.1MB/s]
100% 2.29G/2.29G [00:19<00:00, 125MB/s] 


In [7]:
!mkdir chest-xray-pneumonia

In [ ]:
! unzip /content/chest-xray-pneumonia.zip -d /content/chest-xray-pneumonia

### Pneumonia Detection Using Convolutional Neural Networks in TensorFlow

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Creating the pipelines

In [10]:
def process_img(img, label):
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    return img, label

train_ds = image_dataset_from_directory(
    '/content/chest-xray-pneumonia/chest_xray/train',
    labels='inferred',
    shuffle=True,
    batch_size=64,
    image_size=(256,256),
    color_mode='grayscale',
    label_mode='binary',
    interpolation='nearest'
).map(process_img)

test_ds = image_dataset_from_directory(
    '/content/chest-xray-pneumonia/chest_xray/test',
    labels='inferred',
    shuffle=False,
    batch_size=64,
    image_size=(256,256),
    color_mode='grayscale',
    label_mode='binary',
    interpolation='nearest'
).map(process_img)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


### Making the model

In [11]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(256,256,1)),

    # Data Augmentation
    preprocessing.RandomContrast(0.3),

    # Body
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),

    # Head
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

### Compiling the model

In [12]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_contrast (RandomContr (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 128)       0

### Training the model

In [13]:
early_stopping = EarlyStopping(
    min_delta = 1e-3,
    patience=3,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    train_ds,
    validation_data=(test_ds),
    batch_size=64,
    epochs=10,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/10
82/82 [==============================] - 686s 8s/step - loss: 0.5917 - binary_accuracy: 0.7364 - val_loss: 0.5807 - val_binary_accuracy: 0.6394
Epoch 2/10
82/82 [==============================] - 680s 8s/step - loss: 0.3212 - binary_accuracy: 0.8619 - val_loss: 0.6067 - val_binary_accuracy: 0.7468
Epoch 3/10
82/82 [==============================] - 678s 8s/step - loss: 0.2072 - binary_accuracy: 0.9191 - val_loss: 0.4802 - val_binary_accuracy: 0.8157
Epoch 4/10
82/82 [==============================] - 678s 8s/step - loss: 0.1463 - binary_accuracy: 0.9431 - val_loss: 0.5887 - val_binary_accuracy: 0.8029
Epoch 5/10
82/82 [==============================] - 681s 8s/step - loss: 0.1176 - binary_accuracy: 0.9569 - val_loss: 0.6711 - val_binary_accuracy: 0.7949
Epoch 6/10
82/82 [==============================] - 680s 8s/step - loss: 0.1141 - binary_accuracy: 0.9594 - val_loss: 0.7676 - val_binary_accuracy: 0.7917
Restoring model weights from the end of the best epoch.
Epoch 00006: e

### Saving the model

In [14]:
model.save('pneumonia-model.h5')